In [2]:
%matplotlib inline
import os
import urllib.request


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional
import torch.optim
from sklearn.metrics import accuracy_score
from torchmetrics.functional import precision_recall
import matplotlib
import matplotlib.pyplot as plt
import Bio
from Bio.Seq import Seq

In [4]:
normal_rna = Seq("GCATGCTGCATGCATGATGCCATGCATGCTGCATGCTGCATG")
tumor_rna =  Seq("GCATGCTGCATGCATGATGCCACGCATGCTGCATGCTGCATG")

print(" 野生型RNA配列: .." + str(normal_rna) + "..")
print("腫瘍特異的蛋白質配列: .." + str(tumor_rna) + "..")

print(" 野生型タンパク質配列: .." + str(normal_rna.translate()) + "..")
print("腫瘍特異的タンパク質配列: .." + str(tumor_rna.translate()) + "..")
"""
vcf ファイル上のすべての変異に対して、
変異ペプチドのリストを網羅的に作成すると同時に、
野生型タンパク質配列と比較し、
野生型タンパク質配列にその断片が含まれないことを確認するという手順を実施して
個々の腫瘍が有する体細胞変異や挿入欠失に基づいた、発現しうる腫瘍特異的ペプチドのリストを作成する。
"""

 野生型RNA配列: ..GCATGCTGCATGCATGATGCCATGCATGCTGCATGCTGCATG..
腫瘍特異的蛋白質配列: ..GCATGCTGCATGCATGATGCCACGCATGCTGCATGCTGCATG..
 野生型タンパク質配列: ..ACCMHDAMHAACCM..
腫瘍特異的タンパク質配列: ..ACCMHDATHAACCM..


In [5]:
data_dir = './pepdata/'
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
    
train_url = 'https://github.com/haoqing12/APPM/raw/master/DATA/train_data/A0301'
test_url = 'https://github.com/haoqing12/APPM/raw/master/DATA/test_data/A0301'


train_save_path = os.path.join(data_dir, 'train_A0301.csv')
test_save_path = os.path.join(data_dir, 'test_A0301.csv')

urllib.request.urlretrieve(train_url, train_save_path)
urllib.request.urlretrieve(test_url, test_save_path)

pd.read_csv(train_save_path, nrows=5).head



<bound method NDFrame.head of       Peptide      HLA  BindingCategory
0   GTFGGLGSK  A*03:01                1
1   TTQDPLSNK  A*03:01                1
2  TVAGGAWTYK  A*03:01                1
3  FVYDHVFAEK  A*03:01                1
4   KVTEWQQTY  A*03:01                1>

In [6]:
class PepTransform():
    allSequences = 'ACEDGFIHKMLNQPSRTWVYZ'
    char2int = dict((c, i) for i, c in enumerate(allSequences)) # Add index to amino seq
    
    def peptideOneHotMap(self, peptide):
        peptide_integer = [self.char2int[char] for char in peptide] # convert peptide(str) to int 
        peptide_onehot = list() # peptide seq in Onehot vector format
        for value in peptide_integer:
            letter = [0 for _ in range(len(self.allSequences))]
            
            letter[value] = 1
            peptide_onehot.append(letter)
            
        return np.asarray(peptide_onehot)
        
    def __init__(self):
        '''
        initialize instance variable
        '''
        
    def __call__(self, peptide):
        return self.peptideOneHotMap(peptide)
        